In [1]:
#working with dataset
import pyarrow as pa
import pyarrow.parquet as pq
import re
import string
import pandas as pd
import numpy as np
import re #regular expression
#preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('stopwords')
nltk.download('punkt')
#training the model
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\timber\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\timber\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#load dataset
df = pq.read_table('kinopoisk-train.parquet')
df = pa.Table.to_pandas(df)

In [3]:
#encode categorical values
df.grade3.replace("Good", 1, inplace=True) 
df.grade3.replace("Neutral", 2, inplace=True) 
df.grade3.replace("Bad", 3, inplace=True) 

In [4]:
train_df, test_df = train_test_split(df, test_size=3500) # data split train / test

In [5]:
#data preprocessing
russian_stop_words = stopwords.words("russian")
snowball = SnowballStemmer(language="russian")
def tokenize_sentence(text):
    text = text.replace(r'\n', ' ')
    pattern = "[^A-Za-zА-Яа-яЁё0-9]"
    text = re.sub(pattern," ",text) 
    text = text.lower()
    tokens = word_tokenize(text, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [6]:
#train the logistic regression algorithm    
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer = tokenize_sentence, lowercase=False)),
    ("model", LogisticRegression(random_state=0))])

In [7]:
mp = model_pipeline.fit(train_df["content"], train_df["grade3"])

C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
precision_score(y_true=test_df["grade3"], y_pred=mp.predict(test_df["content"]), average = 'weighted')

In [8]:
def predictor(comment):
    return int(model_pipeline.predict([comment])) #returns 1, 2 or 3

In [ ]:
predictor("Нормально")

In [ ]:
predictor("Ужасный сюжет и актеры, смотреть невозможно")

In [ ]:
predictor("Фильм захватывает с первых секунд")

In [9]:
# for saving algorithm and preprocessing objects
import dill
filename = "mlmodel.pkl"
# save model
dill.dump(mp, open(filename, "wb"))